In [ ]:
import time
import numpy as np
import librosa
from keras import backend as K
import tensorflow as tf
from utils import load_model

_ONE_DAY_IN_SECONDS = 60 * 60 * 24
_PORT = 50052

config=tf.ConfigProto(log_device_placement=False,allow_soft_placement=True)
sess = tf.Session(config=config)
K.set_session(sess) # K is keras backend

In [2]:
model = load_model()

Checking if model is on server.
Model downloaded.
Loading speaker emotion model.
Speaker emotion model succesfully loaded.


In [46]:
import soundfile as sf
(s, sampleRate) = sf.read("./sample_1.wav")


In [47]:
def gen(array, chunkSize):
    i = 0
    length = len(array)
    while i < length:
        chunk = []
        for j in range(i, i + chunkSize):
            if j < length:
                chunk.append(array[j])

        i += chunkSize
        yield (chunk, chunkSize)

In [65]:
it = gen(s, sampleRate)

In [70]:
request_iterator = it
global model, sess
execTime = time.time()

FRAME_LENGTH = 1000
NUMBER_MELS = 128


for req in request_iterator:
    signal, sampleRate = req
    print(f"lensignal {len(signal)}")
    print(f"samplerate {sampleRate}")
    dataset_shape = (FRAME_LENGTH / 10) * NUMBER_MELS
    X_test_vectors = [ np.repeat(0, dataset_shape) ]
    signal = librosa.to_mono(np.transpose(signal))
    trimmedSignal, _ = librosa.effects.trim(signal, top_db=50)
    spectrogram = librosa.feature.melspectrogram(trimmedSignal, sr=sampleRate, n_fft=1024, hop_length=160)
    logSpectrogram = refFun(spectrogram)

    signalLength = float(len(trimmedSignal) / sampleRate) * 1000
    indexPosition = 0
    print(signalLength)
    while indexPosition < signalLength - FRAME_LENGTH:
        row = np.asarray(logSpectrogram[:, int(0 / 10):int((0 + FRAME_LENGTH) / 10)]).ravel()
        X_test_vectors.append(row)
        indexPosition += FRAME_LENGTH

    X_test_vectors = X_test_vectors[1:]
    X_test = []
    print(f"len{len(X_test_vectors)}")
    for i in range(len(X_test_vectors)):
        matrix = np.zeros((NUMBER_MELS, int(FRAME_LENGTH / 10)))
        for l in range(NUMBER_MELS):
            for m in range(int(FRAME_LENGTH / 10)):
                matrix[l, m] = X_test_vectors[i][l * int(FRAME_LENGTH / 10) + m]
        
        X_test.append([matrix])

    with sess.graph.as_default():
        predict = model.predict(X_test)
    print(predict)

    # yield speaker_emotion_pb2.Response(
    # neutral=predict[-1][0],
    # calm=predict[-1][1],
    # happy=predict[-1][2],
    # sad=predict[-1][3],
    # angry=predict[-1][4],
    # fearful=predict[-1][5],
    # surprise=predict[-1][6],
    # disgust=predict[-1][7],
    # exec_time=execTime)
    execTime = time.time() - execTime

lensignal 44000
samplerate 44000
1000.0
len0


ValueError: Error when checking : expected conv2d_1_input to have 4 dimensions, but got array with shape (0, 1)

In [22]:
def refFun(S):
    return np.log10(1 + 10000 * S)